In [3]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.preprocessing import StandardScaler

In [4]:
from sklearn.datasets import fetch_openml
# X, y = fetch_openml('mnist_784', version=1, return_X_y=True)
X, y = load_digits(return_X_y=True)


In [66]:
y = y.astype(np.int32)

In [67]:
y = np.eye(10)[y]

In [68]:
X = StandardScaler().fit_transform(X)


In [69]:
X.shape, y.shape

((70000, 784), (70000, 10))

In [131]:
x = np.array([[1, 2, 3, 4], [4, 3, 2, 1], [1, 2, 3, 4], [4, 3, 2, 1]])

In [132]:
x = np.repeat(x[..., np.newaxis], 3, axis=-1)

In [133]:
x = np.repeat(x[np.newaxis, ...], 8, axis=0)

In [134]:
x.shape

(8, 4, 4, 3)

In [135]:
x = np.pad(x, [(0, 0), (1, 1), (1, 1), (0, 0)])

In [136]:
k = 3

In [137]:
kernel = np.array([[0] * 9 * 3, [1] * 9 * 3, [2] * 9 * 3, [3] * 9 * 3]).T

In [138]:
kernel

array([[0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3]])

In [139]:
slices = []

for i in range(0, x.shape[1] - k + 1):
    for j in range(0, x.shape[2] - k + 1):
        print(x[:, i: i + k, j: j + k, ...].shape)
        slices.append(np.reshape(x[:, i: i + k, j: j + k, ...], [x.shape[0], -1]))

(8, 3, 3, 3)
(8, 3, 3, 3)
(8, 3, 3, 3)
(8, 3, 3, 3)
(8, 3, 3, 3)
(8, 3, 3, 3)
(8, 3, 3, 3)
(8, 3, 3, 3)
(8, 3, 3, 3)
(8, 3, 3, 3)
(8, 3, 3, 3)
(8, 3, 3, 3)
(8, 3, 3, 3)
(8, 3, 3, 3)
(8, 3, 3, 3)
(8, 3, 3, 3)


In [140]:
slices = np.stack(slices, axis=1)
slices.shape

(8, 16, 27)

In [141]:
(slices @ kernel).shape # B, W x H, C_out

(8, 16, 4)

In [142]:
out = slices @ kernel

In [143]:
c_out = 4

In [144]:
out = out.reshape([out.shape[0], 4, 4, c_out])

In [145]:
out.shape

(8, 4, 4, 4)

In [217]:
class Conv:
    def __init__(self, in_dim, out_dim, k=3, padding=1):
        self.w = np.random.randn(k * k * in_dim, out_dim)
        self.b = np.zeros([out_dim])
        self.dw = None
        self.db = None
        self.x = None
        
        self.padding = padding
        self.out_dim = out_dim
        
    def forward(self, x):
        self.x = x
        s = x.shape
        x = np.pad(x, [(0, 0), (self.padding, self.padding), (self.padding, self.padding), (0, 0)])
        slices = np.stack([np.reshape(x[:, i: i + k, j: j + k, ...], [s[0], -1]) for i in range(0, s[1] - k + 1 +  2 * self.padding) for j in range(0, s[2] - k + 1  + 2 * self.padding)], axis=1)
        x = slices @ self.w
        return np.reshape(x, [s[0], s[1], s[2], self.out_dim]) + self.b
        
    def backward(self, d):
        self.dw = np.matmul(self.x.T, d)
        self.db = np.sum(d, axis=0)
        assert self.dw.shape == self.w.shape
        assert self.db.shape == self.b.shape

        d = d @ self.w.T
        return d
        
    def step(self, lr):
        self.w = self.w - lr * self.dw
        self.b = self.b - lr * self.db

In [218]:
conv = Conv(3, 8)

In [219]:
conv.forward(np.random.randn(8, 4, 4, 3))

array([[[[ 1.20359032e+00,  3.83362559e+00, -4.18603987e-01, ...,
          -1.16018147e-01, -5.30471366e-01,  6.68043440e-02],
         [-5.44385535e-01,  6.03769946e-01,  1.26996167e+00, ...,
          -3.02069555e+00, -5.24234575e+00,  7.27136364e+00],
         [ 2.11526199e-01,  1.73746751e+00, -3.06965721e+00, ...,
          -3.86698869e+00,  4.28950854e+00,  6.20486783e-01],
         [ 3.05478549e+00, -3.76903748e+00,  3.09160261e+00, ...,
          -1.98284794e+00, -5.59170528e-01,  1.20154283e+00]],

        [[-3.54928645e+00, -6.08209743e+00,  8.14016369e-01, ...,
           2.55577556e+00, -4.05574651e+00,  6.03798041e-01],
         [ 1.60979231e+00,  4.37581324e-02, -3.01933435e+00, ...,
           1.12614889e+00,  7.98736115e-01, -3.18866561e+00],
         [-3.56655571e+00, -4.87557861e+00,  4.09567296e+00, ...,
           5.45331793e+00, -4.32981873e+00, -2.35138241e+00],
         [-4.44562374e-01,  5.66597170e+00, -7.36590164e+00, ...,
           7.42370900e+00, -2.684108

In [70]:
class Linear:
    def __init__(self, in_dim, out_dim):
        self.w = np.random.randn(in_dim, out_dim)
        self.b = np.zeros([out_dim])
        self.dw = None
        self.db = None
        self.x = None
        
    def forward(self, x):
        self.x = x
        return np.matmul(x, self.w) + self.b
    
    def backward(self, d):
        self.dw = np.matmul(self.x.T, d)
        self.db = np.sum(d, axis=0)
        assert self.dw.shape == self.w.shape
        assert self.db.shape == self.b.shape

        d = d @ self.w.T
        return d
        
    def step(self, lr):
        self.w = self.w - lr * self.dw
        self.b = self.b - lr * self.db

In [71]:
class ReLU:
    def __init__(self):
        self.a = None
        
    def forward(self, x):
        self.a = np.maximum(x, 0)
        return self.a
    
    def backward(self, d):
        return d * (self.a != 0).astype(np.float32)
        

In [72]:
class Softmax:
    def __init__(self):
        self.a = None
        
    def forward(self, x):
        assert len(x.shape) == 2
        x = x - np.max(x, axis=-1, keepdims=True)
        self.a = np.exp(x) / np.sum(np.exp(x), keepdims=True, axis=-1)
        return self.a
    def backward(self, d):
        sigma = np.repeat(self.a[:, np.newaxis, ...], self.a.shape[1], 1)
        j = np.swapaxes(sigma, 1, 2) * (np.eye(sigma.shape[1])[np.newaxis, ...] - sigma)
        return (j @ d[..., np.newaxis]).squeeze(-1)


In [73]:
class CrossEntropy:
    def __init__(self, average=True): 
        self.average = average
    def forward(self, y_, y):
        
        l = - np.sum(y * np.log(y_ + 1e-9))
        if self.average:
            l /= len(y)
        return y_, l
    
    def backward(self, y_, y):
        assert y_.shape == y.shape
        d =  - y / (y_ + 1e-9)
        if self.average:
            d /= len(y)
        return d

In [89]:
class LogisticRegression:
    def __init__(self):
        self.linear1 = Linear(784, 128)
        self.relu1 = ReLU()
        self.linear2 = Linear(128, 32)
        self.relu2 = ReLU()
        self.linear3 = Linear(32, 10)
        self.softmax = Softmax()
        self.loss = CrossEntropy()
        
    def forward(self, x, y):
        x = self.linear1.forward(x)
        x = self.relu1.forward(x)
        x = self.linear2.forward(x)
        x = self.relu2.forward(x)
        x = self.linear3.forward(x)
        x = self.softmax.forward(x)
        loss = self.loss.forward(x, y)
        return loss
    
    def backward(self, y_, y):
        d = self.loss.backward(y_, y)
        d = self.softmax.backward(d)
        d = self.linear3.backward(d)
        d = self.relu2.backward(d)
        d = self.linear2.backward(d)
        d = self.relu1.backward(d)
        d = self.linear1.backward(d)

    
    def step(self, lr):
        self.linear1.step(lr)
        self.linear2.step(lr)
        self.linear3.step(lr)

        

In [93]:
logreg = LogisticRegression()

In [94]:
batch_size = 1024

In [95]:
for i in range(10000):
    for j in range(0, len(X), batch_size):
        X_batch = X[j: j + batch_size]
        y_batch = y[j: j + batch_size]
        y_, loss = logreg.forward(X_batch, y_batch)
        logreg.backward(y_, y_batch)
        logreg.step(lr=0.01)
        if j % 32768 == 0:
            print(j)
            print(loss, 'loss')
            print((np.argmax(y_, axis=-1) == np.argmax(y_batch, axis=-1)).sum() / len(y_batch))

0
18.496505292805416 loss
0.107421875
32768
16.946182116003918 loss
0.177734375
65536
16.086337211345644 loss
0.21875
0
17.10940405439259 loss
0.169921875
32768
16.15429415927803 loss
0.216796875
65536
15.39941509923963 loss
0.2529296875
0
16.769827927237053 loss
0.185546875
32768
15.674775865435535 loss
0.240234375
65536
15.167747661812076 loss
0.265625
0
16.058790743877857 loss
0.2197265625
32768
15.101982019679099 loss
0.267578125
65536
14.941254522515113 loss
0.27734375
0
15.553179781141889 loss
0.2412109375
32768
14.760180221880498 loss
0.2861328125
65536
14.398715705364193 loss
0.3037109375
0
15.133704714364086 loss
0.2685546875
32768
14.359092710676993 loss
0.3056640625
65536
14.21783799967715 loss
0.3134765625
0
14.759393980766395 loss
0.283203125
32768
13.871345565210346 loss
0.328125
65536
13.840032218927705 loss
0.326171875
0
14.104736425201262 loss
0.3173828125
32768
13.520520716456282 loss
0.34375
65536
13.458752735508272 loss
0.345703125
0
13.759195960392509 loss
0.332031

KeyboardInterrupt: 